
# Create a four chamber heart model
This example shows you how to download a case from the Strocchi et al (2020) database
and process that case into a simulation-ready four chamber heart model.


# Example setup
Perform the required imports
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Import the required modules and set relevant paths, including that of the working
directory and generated model



In [ ]:
import os
from pathlib import Path

from ansys.heart.misc.downloader import download_case, unpack_case
import ansys.heart.preprocessor.models as models


# specify necessary paths.
# Note that we need to cast the paths to strings to facilitate serialization.
case_file = str(
    Path(Path(__file__).resolve().parents[2], "downloads", "Strocchi2020", "01", "01.case")
)
download_folder = str(Path(Path(__file__).resolve().parents[2], "downloads"))
workdir = str(
    Path(Path(__file__).resolve().parents[2], "downloads", "Strocchi2020", "01", "FourChamber")
)
path_to_model = str(Path(workdir, "heart_model.pickle"))

## Download the case
Download and unpack the case from the public repository of full hearts if it is
not yet available. This model will be used as input for the preprocessor.



In [ ]:
if not os.path.isfile(case_file):
    path_to_downloaded_file = download_case(
        "Strocchi2020", 1, download_folder=download_folder, overwrite=False
    )
    unpack_case(path_to_downloaded_file)

## Set required information
Set the right database to which this case belongs, and set other relevant
information such as the desired mesh size.



In [ ]:
info = models.ModelInfo(
    database="Strocchi2020",
    path_to_case=case_file,
    work_directory=workdir,
    path_to_model=path_to_model,
    add_blood_pool=False,
    mesh_size=1.5,
)

# create the working directory
info.create_workdir()
# clean the working directory
info.clean_workdir(extensions_to_remove=[".stl", ".vtk", ".msh.h5"])
# dump information to stdout
info.dump_info()

## Initialize the heart model
Initialize the desired heart model, and invoke the main method to
extract the simulation mesh and dump the model to disk. This will extract
the relevant parts from the original model and remesh the entire surface and
volume. Moreover, relevant anatomical features are extracted.



In [ ]:
# instantiate a four chamber model
model = models.FourChamber(info)

# extract the simulation mesh
model.extract_simulation_mesh()

# dump the model to disk for future use
model.dump_model(path_to_model)
# print the resulting information
model.print_info()

# print part names
print(model.part_names)
# print volumes of some cavities:
print(f"Volume of LV cavity: {model.left_ventricle.cavity.volume} mm^3")
print(f"Volume of LV cavity: {model.left_atrium.cavity.volume} mm^3")

## Visualize results
You can visualize and inspect the components of the model by accessing
various properties/attributes and invoke methods.



In [ ]:
# plot the remeshed model
model.plot_mesh(show_edges=False)

<img src="file://_static/images/four_chamber_mesh.png" width="400pt" align="center">



In [ ]:
# plot the endocardial surface of the left ventricle.
model.left_ventricle.endocardium.plot(show_edges=True, color="r")

<img src="file://_static/images/four_chamber_lv_endocardium.png" width="400pt" align="center">



In [ ]:
# loop over all cavities and plot these in a single window.
import pyvista as pv

cavities = pv.PolyData()
for c in model.cavities:
    cavities += c.surface
cavities.plot(show_edges=True)

<img src="file://_static/images/four_chamber_cavities.png" width="400pt" align="center">

